<a href="https://colab.research.google.com/github/dovkess/FGCVBreedDetection/blob/main/trainDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import cell. this is a cell run once to avoid having import problems later on.
%%capture
!pip install tensorflow
from google.colab import drive
import sys, pickle
import time
import numpy as np
import glob
import os
from tensorflow.keras.models import load_model
drive.mount('/content/drive')
name = 'dog_functions'
new_name = 'dog_funcs'
!jupyter nbconvert /content/drive/MyDrive/Colab_Notebooks/{name}.ipynb --to python
%rm /content/drive/MyDrive/Colab_Notebooks/{new_name}.py
%mv /content/drive/MyDrive/Colab_Notebooks/{name}.py /content/drive/MyDrive/Colab_Notebooks/{new_name}.py
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/')
import dog_funcs
# num_dogs_by_breed_loaded = pickle.load(open('/content/drive/MyDrive/DogProject/class_indices_2025-10-14_21:50:22.pkl', 'rb'))

In [ ]:
# load folds seperation.
# define locations

folds = pickle.load(open('/content/drive/MyDrive/DogProject/oxford_ds/folds_sep', 'rb'))
cropped_loc = '/content/drive/MyDrive/DogProject/oxford_ds/cropped_all_images'
images_loc = '/content/drive/MyDrive/DogProject/oxford_ds/images'
snapshot_loc = '/content/drive/MyDrive/DogProject/oxford_ds/snapshots'
classes = set([os.path.dirname(folds[0]['train'][i]) for i in range(len(folds[0]['train']))])
print(os.path.isdir(cropped_loc))
print(os.path.isdir(images_loc))

True
True


In [ ]:
# load classes for prediction
classes = pickle.load(open('/content/drive/MyDrive/DogProject/oxford_ds/class_indices_12_28_21_39.pkl', 'rb'))
classes = list(classes.keys())

In [ ]:
###### ==== RUN ONE OF K FOLD DATA ==== ######
# run for each fold i
i=2
fold_ind = i
train_list = folds[i]['train']
test_list = folds[i]['test']
validation_list = folds[i]['validation']

# copy images -- (cropped, uncropped) for (test, validation, train)
dog_funcs.copy_images_by_list(validation_list, cropped_loc, local_loc='/{}/cropped/'.format(i))
dog_funcs.copy_images_by_list(validation_list, images_loc, local_loc='/{}/not_cropped'.format(i))

dog_funcs.copy_images_by_list(train_list, cropped_loc, local_loc='/{}/cropped_train'.format(i))
dog_funcs.copy_images_by_list(train_list, images_loc, local_loc='/{}/not_cropped_train'.format(i))

dog_funcs.copy_images_by_list(test_list, cropped_loc, local_loc='/{}/cropped_validation'.format(i))
dog_funcs.copy_images_by_list(test_list, images_loc, local_loc='/{}/not_cropped_validation'.format(i))
time.sleep(15)


2026-01-02_16:37:09
2026-01-02_16:37:19
2026-01-02_16:37:19
2026-01-02_16:37:29
2026-01-02_16:37:29
2026-01-02_16:38:25
2026-01-02_16:38:25
2026-01-02_16:39:21
2026-01-02_16:39:21
2026-01-02_16:39:36
2026-01-02_16:39:36
2026-01-02_16:39:51


In [ ]:
import time
##### ===== LOAD AND TRAIN SINGLE MODEL ===== #######
def single_model(base_data_path, fold, name, input_size, epochs, model_function, snapshot_loc, classes, lr=None, batch_size=128):
    '''
    Train and predict a single model. (transfer learn).

    Args:
        base_data_path (str): location of images.
        fold (int): number of fold -- set in previous cell
        name (str): model name
        input_size (list): [width, height]
        epochs (int): number of epochs to train the net
        model_function (func): get relevant model
        snapshot_loc (str): location to save snapshots of training
        classes (list): list of labels
        lr (float): learning rate
        batch_size (int): batch size for training

    Return:
        acc (float): accuracy for not cropped images
        acc_cropped (float): accuracy for cropped images
        model (keras model): model trained on not cropped images
        model_cropped (keras model): model trained on cropped images
        validation_data (keras data generator): validation data set
        validation_data_cropped (keras data generator): validation data set cropped
    '''
    # prepare data set
    train_data = dog_funcs.create_single_data_set('{}/not_cropped_train'.format(base_data_path), batch_size, classes, input_size=input_size)
    test_data = dog_funcs.create_single_data_set('{}/not_cropped'.format(base_data_path), batch_size, classes, input_size=input_size)
    validation_data = dog_funcs.create_single_data_set('{}/not_cropped_validation'.format(base_data_path), batch_size, classes, input_size=input_size)

    train_data_cropped = dog_funcs.create_single_data_set('{}/cropped_train'.format(base_data_path), batch_size, classes, input_size=input_size)
    test_data_cropped = dog_funcs.create_single_data_set('{}/cropped'.format(base_data_path), batch_size, classes, input_size=input_size)
    validation_data_cropped = dog_funcs.create_single_data_set('{}/cropped_validation'.format(base_data_path), batch_size, classes, input_size=input_size)

    # snapshot location
    snapshots_path = '{}/folds_{}_{}_{}'.format(snapshot_loc, fold, name, time.strftime("%m_%d_%H_%M"))
    snapshots_path = snapshots_path + '{}cropped/'

    # Train model:
    model = model_function(len(classes), lr=lr)
    snapshots_path_not = snapshots_path.format('_not_')
    dog_funcs.train_net(model, train_data, test_data,
                        snapshot_dir=snapshots_path_not, epochs=epochs)
    # Load best checkpoint
    w = glob.glob(snapshots_path_not+'/check*.keras')[0]
    model = load_model(w)
    # predict results
    acc, e_idx = dog_funcs.predict_accuracy(model, validation_data)
    print('Test accuracy {} not cropped: {}'.format(name, acc))

    # Train model
    model_cropped = model_function(len(classes), lr=lr)
    snapshots_path_cropped = snapshots_path.format('_')
    dog_funcs.train_net(model_cropped, train_data_cropped, test_data_cropped,
                        snapshot_dir=snapshots_path_cropped, epochs=epochs)

    # Load best checkpoint
    w = glob.glob(snapshots_path_cropped+'/check*.keras')[0]
    model_cropped = load_model(w)

    # predict results
    acc_cropped, e_idx = dog_funcs.predict_accuracy(model_cropped, validation_data_cropped)
    print('Test accuracy {} cropped: {}'.format(name, acc_cropped))

    return acc, acc_cropped, model, model_cropped, validation_data, validation_data_cropped


In [ ]:
# NAS model for training
acc_nas, acc_cropped_nas, m, m_c, d, d_c = single_model('/content/{}/'.format(i), i, 'NAS', input_size=[331, 331],
                                        epochs=30, model_function=dog_funcs.define_nasnetlarge_net,
                                        snapshot_loc=snapshot_loc, classes=classes)

# inseption model for training
acc_inseption, acc_cropped_inseption, m, m_c, d, d_c = single_model('/content/{}/'.format(i), i,
                                                    'inseption', input_size=[229, 229], epochs=40,
                                                    model_function=dog_funcs.define_inseption_net,
                                                    snapshot_loc=snapshot_loc, classes=classes)
# ResNet152 model for training
acc_resnet152, acc_cropped_resnet152, m, m_c, d, d_c = single_model('/content/{}/'.format(i), i, 'ResNet152',
                                                    input_size=[224, 224], epochs=40,
                                                    model_function=dog_funcs.define_resnet152_net,
                                                    snapshot_loc=snapshot_loc, classes=classes, lr=0.001)
# Xception model for training
acc_xception, acc_cropped_xception, m, m_c, d, d_c = single_model('/content/{}/'.format(i), i, 'Xception',
                                                    input_size=[299, 299], epochs=40,
                                                    model_function=dog_funcs.define_xception_net,
                                                    snapshot_loc=snapshot_loc, classes=classes)
# DenseNet201 model for training
acc_densenet201, acc_cropped_densenet201, m, m_c, d, d_c = single_model('/content/{}/'.format(i), i, 'DenseNet201',
                                                        input_size=[224, 224], epochs=40,
                                                        model_function=dog_funcs.define_densenet201_net,
                                                        snapshot_loc=snapshot_loc, classes=classes)


Found 5320 images belonging to 37 classes.
Found 592 images belonging to 37 classes.
Found 1478 images belonging to 37 classes.
Found 5320 images belonging to 37 classes.
Found 592 images belonging to 37 classes.
Found 1478 images belonging to 37 classes.
343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ nasnet_large (Functional)       │ (None, 11, 11, 4032)   │    84,916,818 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 4032)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 37)             │       149,221 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,066,039 (324.50 MB)

 Trainable params: 149,221 (582.89 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet201 (Functional)        │ (None, 7, 7, 1920)     │    18,321,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_9      │ (None, 1920)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 37)             │        71,077 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,393,061 (70.16 MB)

 Trainable params: 71,077 (277.64 KB)

 Non-trainable params: 18,321,984 (69.89 MB)

Epoch 1/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.0051 - loss: 5.2120 - val_accuracy: 0.1926 - val_loss: 3.0961
Epoch 2/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 15s 347ms/step - accuracy: 0.1256 - loss: 3.2084 - val_accuracy: 0.4544 - val_loss: 2.1496
Epoch 3/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 307ms/step - accuracy: 0.3912 - loss: 2.2110 - val_accuracy: 0.6351 - val_loss: 1.5448
Epoch 4/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 305ms/step - accuracy: 0.5532 - loss: 1.6853 - val_accuracy: 0.6909 - val_loss: 1.2328
Epoch 5/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 310ms/step - accuracy: 0.7359 - loss: 1.1671 - val_accuracy: 0.7889 - val_loss: 0.9704
Epoch 6/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 313ms/step - accuracy: 0.7041 - loss: 1.0920 - val_accuracy: 0.7922 - val_loss: 0.8383
Epoch 7/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 303ms/step - accuracy: 0.8157 - loss: 0.7830 - val_accuracy: 0.8328 - val_loss: 0.7255
Epoch 8/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 305ms/step - accuracy: 0.7493 - loss: 0.8831 - val_accura

In [ ]:
# save results
acc_dict = {'NAS': {'acc_nas': acc_nas, 'acc_cropped_nas': acc_cropped_nas},
            'INSEPTION': {'acc_inseption': acc_inseption, 'acc_cropped_inseption': acc_cropped_inseption},
            'RESNET152': {'acc_resnet152': acc_resnet152, 'acc_cropped_resnet152': acc_cropped_resnet152},
            'XCEPTION': {'acc_xception': acc_xception, 'acc_cropped_xception': acc_cropped_xception},
            'DENSENET201': {'acc_densenet201': acc_densenet201, 'acc_cropped_densenet201': acc_cropped_densenet201}
            }

pickle.dump(acc_dict, open('/content/drive/MyDrive/DogProject/oxford_ds/acc_dict_fold_{}.pkl'.format(fold_ind), 'wb'))